In [1]:
## Preamble: Package Loading
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import itertools as iter
import math

<h4> Block Diagonal Matrix Function </h4>

In [2]:
def blkdiag(mat,nb):
    """
INPUTS
mat     Square Matrix which will form the block in a block diagonal matrix
nb      Number of diagonal block in output matrix 

OUTPUT
v       Block diagonal matrix of dimension ( nb*mat.shape[0] x nb*mat.shape[0] )
    """
    # Initializing the varcov matrix for all crosssections
    v = np.hstack((mat,np.zeros((mat.shape[0],(nb-1)*(mat.shape[1])))))
    # Registry matrix used in following loops
    vreg = np.eye(nb-1)
    for j in np.arange(nb-1):
        # Initializing current block of rows 
        pv = np.zeros((mat.shape[0],mat.shape[0]))
        # Horiz Stacking either zeros or var_err depending on ve_reg[j,i]
        for i in np.arange(nb-1):
            if vreg[j,i] == 1: # Stack var_err onto pv_err
                pv = np.hstack((pv,mat))
            if vreg[j,i] == 0: # Stack zeros onto pv_err
                pv = np.hstack((pv,np.zeros((mat.shape[0],mat.shape[0]))))
        # Vertically stacking block rows on top of one another
        v = np.vstack((v,pv))
    return v

#### Block Diagonal Matrix Function Demonstration

In [3]:
blkdiag(np.ones((3,3)),4)

array([[ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.]])

<h3> DGP Inputs </h3>

In [4]:

# Setting a Seed
np.random.seed([10])

# Number of Time Periods
ntp = 10
# Number of Cross-Sections
ncs = 3
# Number of Endogenous Variables in Primary Equation
n_end = 2
# Number of Endogenous Variables in Primary Equation
n_exo = 2
# Total Number of Instruments
t_inst = 10
# Number of Instruments per Crossection
c_inst = 3
# Indicator for whether to force additive non linear cntrl function. 1 = yes 
frc = 0

# Vector of exog off diagonal covariances i.e. cov(Z2t_l,Z2t_(l+j)) = ex_vpro[j-1]
ex_vpro = [0.5]
# Vector of inst off diagonal covariances i.e. cov(Wt_l,Wt_(l+j)) = inst_vpro[j-1]
inst_vpro = [0.5 , 0.25 ]
# Vector of error off diagonal covariances i.e. cov(Wt_l,Wt_(l+j)) = inst_vpro[j-1]
err_vpro = [0.8 , 0.36 ]

<h2> DGP Generation Code </h2>

In [77]:

# Collection of all Coefficients on Instruments 
icoeffs_reg = list(iter.product([-1,1],repeat = c_inst))
# Collection of all Coefficients on Exogenous Variables
excoeffs_reg = list(iter.product([-1,1],repeat = n_exo))
# Registry of instrument assignments
insts_reg = list(iter.combinations(np.arange(1,t_inst+1),c_inst))

# List of which instruments (col #'s) are used for each crossection 
icr = [insts_reg[np.random.randint(len(insts_reg))] for i in range(ncs)]

# All permutation of combinations of n_end icoefficient vectors  
picfs = list(iter.permutations(range(0,len(icoeffs_reg)),n_end))
icfs = [ picfs[i] for i in np.random.randint(len(picfs),size = ncs)]

# All permutation of combinations of n_end icoefficient vectors  
pxcfs = list(iter.permutations(range(0,len(excoeffs_reg)),n_end))
xcfs = [ pxcfs[i] for i in np.random.randint(len(pxcfs),size = ncs)]
            
# Initializing Coefficient Matrix            
coeff = np.zeros((n_end,ncs,t_inst+n_exo))
for j in range(n_end):
    for i in range(ncs):
        for k in range(n_exo):
            coeff[j,i,k] = excoeffs_reg[xcfs[i][j]][k]
        
        for k in range(c_inst): 
            coeff[j,i,icr[i][k]+n_exo-1] = icoeffs_reg[icfs[i][j]][k]            
            
## Joint Distribution of Exogenous regressors
# Vector of Means (=0)
mu_ex = np.zeros(n_exo)
# Diagonal matrix of variaces (=1)
var_ex = np.eye(n_exo)
# Variance Covariance Matrix Generation or EACH crossection
for i in np.arange(len(ex_vpro)):
    var_ex = (var_ex + ex_vpro[i]*np.eye(n_exo,k=i+1)
                         + ex_vpro[i]*np.eye(n_exo,k=-(i+1))) 
# Exogenous regressor mean vector for ALL Crossections
Mu_ex = np.tile(mu_ex,ncs) 
# Exogenous regressor variance covariance matrix for ALL Crossections
V_ex = blkdiag(var_ex,ncs)
    
## Joint Distribution of Instruments for all cross sections
# Vector of Means (=0)
mu_inst = np.zeros(t_inst)
# Diagonal Matrix of Variaces (=1)
var_inst = np.eye(t_inst)
# Variance Covariance Matrix Generation
for i in np.arange(len(inst_vpro)):
    var_inst = (var_inst + inst_vpro[i]*np.eye(t_inst,k=i+1) 
                         + inst_vpro[i]*np.eye(t_inst,k=-(i+1)))
    
## Joint Distribution of Error Terms for EACH crossection
# Vector of means
mu_err = np.zeros(n_end+1)
# Diagonal Matrix of Variances 
var_err = np.eye(n_end+1)
# Variance Covariance Matrix Generation
if frc == 0 : 
    # Var Cov matrix for correlated errors ==> additive linear control functions 
    for i in np.arange(len(err_vpro)):
        var_err = (var_err + err_vpro[i]*np.eye(n_end+1,k=i+1) 
                             + err_vpro[i]*np.eye(n_end+1,k=-(i+1)))
# Error term mean vector for ALL Crossections
Mu_err = np.tile(mu_err,ncs)        
# Error term variance covariance matrix for ALL Crossections
V_err = blkdiag(var_err,ncs)

## Variable Generation
# Exogenous Regressor Generation
Ex = np.random.multivariate_normal(Mu_ex,V_ex,ntp)
# Instruments Generation
Inst = np.random.multivariate_normal(mu_inst,var_inst,ntp)
# Error Terms Generation 
Err = np.random.multivariate_normal(Mu_err,V_err,ntp)

## Variable Name Generation
ex_nms = [''.join(['Z2',str(i),',',str(j)]) for i in list(range(1,ncs+1)) for j in list(range(1,n_exo+1))]

# Constructing names for instruments
inst_nms = [''.join(['W',str(i)]) for i in list(range(1,t_inst+1))]

# Constructing names for error terms
err_nm1 = ['e' if val == n_end+1 else 'V' for val in  list(range(1,n_end+2))*ncs]
err_nm2 = [ str(i) for y in range(1,n_end+2) for i in iter.repeat(y,n_end+1)]
err_nm3 = ['' if val == n_end+1 else ''.join([',',str(val)]) for val in list(range(1,n_end+2))*ncs]
err_nm  = [''.join([err_nm1[i],err_nm2[i],err_nm3[i]]) for i in range(len(err_nm1))]

## Optional Data Frame Generation 
Ex_df = pd.DataFrame(Ex,columns = ex_nms)
Inst_df = pd.DataFrame(Inst, columns = inst_nms)
Err_df = pd.DataFrame(Err,columns = err_nm)


for j in range(n_end):
    for i in range(ncs):
        ex_pat = ''.join(['^Z2',str(i+1)])
        err_pat = ''.join(['V',str(i+1),',',str(j+1)])
        pe1 = pd.concat([Ex_df.filter(regex = ex_pat),Inst_df], axis = 1).dot(coeff[j,i,:].T) 
        pe2 = Err_df.filter(regex = err_pat)
        
        end_nm = ''.join(['Z1',str(i+1),',',str(j+1)])
        if j == 0 and i == 0:
            End_df = pd.DataFrame(pe , columns = end_nm)
        else:
            End_df[end_nm] = pe


TypeError: Index(...) must be called with a collection of some kind, 'Z11,1' was passed

In [89]:
np.array(pd.concat([Ex_df.filter(regex = ex_pat),Inst_df], axis = 1).dot(coeff[j,i,:].T)) 

array([-3.63338872,  1.41770408, -3.34032752, -1.24918327,  5.47122377,
        1.05816222, -7.63850678,  0.18764497,  0.15319794, -0.38839044])

In [88]:
A.shape

(10, 10)

In [59]:
# Crossection
i = 1
ex_pat = ''.join(['^Z2',str(i)])
Ex_df.filter(regex = ex_pat)

,"Z21,1","Z21,2"
0,-1.185911,-0.157085
1,-0.564525,-1.208690
2,-1.216010,-0.389699
3,-0.441434,-1.638212
4,0.855335,1.110917
5,0.055662,-1.818122
6,0.699416,-0.118647
7,0.303488,-0.773107
8,-0.071476,-0.984908
9,0.142248,1.161866


In [66]:
# Element of V
j = 1
err_pat = ''.join(['V',str(i),',',str(j)])
Err_df.filter(regex = err_pat)

,"V1,1"
0,-1.069342
1,0.240064
2,0.221990
3,-0.456494
4,-2.098799
5,0.657157
6,1.181719
7,0.274723
8,-0.664812
9,-0.791027


In [ ]:
coeff = np.zeros((n_end,ncs,t_inst+n_exo))

for j in range(n_end):
    for i in range(ncs):
        for k in range(n_exo):
            coeff[j,i,k] = excoeffs_reg[xcfs[i][j]][k]
        
        for k in range(c_inst): 
            coeff[j,i,cr[i][k]+n_exo-1] = icoeffs_reg[icfs[i][j]][k]
            


In [ ]:
var_inst

In [ ]:
V_ex

In [ ]:
V_err